In [2]:
import os
import numpy as np
from nltk.corpus import stopwords
from string import punctuation
from gensim.models import word2vec

import pandas as pd
import numpy as np

In [18]:
train=pd.read_csv("r8-train.csv")
test=pd.read_csv("r8-test.csv")

In [19]:
train.head(5)

,text,edge,intent
0,champion product approv stock split champion p...,champion product approv stock split champion p...,earn
1,comput termin system cpml complet sale comput ...,comput termin system cpml complet sale comput ...,acq
2,cobanco inc cbco year net shr ct dlr net asset...,cobanco inc cbco year net shr ct dlr net asset...,earn
3,intern inc qtr jan oper shr loss two ct profit...,intern inc qtr jan oper shr loss two ct profit...,earn
4,brown forman inc bfd qtr net shr dlr ct net ml...,brown forman inc bfd qtr net shr dlr ct net ml...,earn


#for text files-reading data
X,y=[],[]
with open(train,'r') as infile:
    for line in infile:
        label,text = line.split("\t")
        X.append(text.split())
        y.append(label)
X,y=np.array(X),np.array(y)
print("total example %s" % len(y))

In [21]:
X=train.drop(["intent"],axis=1)
y=train["intent"]

In [22]:
X_test=test.drop(["intent"],axis=1)
y_test=test["intent"]

In [24]:
print("total example of train %s" % len(y))
print("total example of test %s" % len(y_test))

total example of train 4937
total example of test 2189


# Feature engineering

In [25]:
#finding unique labels available with us
np.unique(y,return_counts=True)

(array(['acq', 'crude', 'earn', 'grain', 'interest', 'money-fx', 'ship',
        'trade'], dtype=object),
 array([1425,  237, 2610,   32,  132,  178,   95,  228], dtype=int64))

these are the unique labels

# Navy bias classification



In [ ]:
#x data all words are concatenated
X_text=[" ".join(val) for val in X]
print(X_test[0]+"\n")

X_test_text=[" ".join(val) for val in X_test]
print(X_test_text[0]+"\n")

# converting text to vectors and removing stop words'



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect=CountVectorizer(stop_words='english',max_features=5000)   #5000 different features is dervied 
vect.fit(X_text)

Applying the vectorizer to our test and train sets

In [ ]:
X_train_transformed=vect.transform(X_text)
X_test_transformed=vect.transform(X_test_text)

In [ ]:
#printing the vocabularoy
list(vect.vocabulory_.itms())[:10]

# BERNOUILLI NB

In [ ]:
from sklearn.metrics import accuracy_score

from sklearn.naive_bayes import BernoulliNB
bnb=BernoulliNB()
bnb.fit(X_train_transformed,y)

#predict class
pred_train_ys=bnb.predict(X_train_transformed)
pred_test_ys=bnb.predict(X_test_transformed)

#accuracy
print("Train accuracy", accuracy_score(y,pred_train_ys))
print("Test accuracy", accuracy_score(y,pred_test_ys))

# MULTINOMIAL NB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()

mnb.fit(X_train_transformed,y)

#predict class
pred_train_ys=bnb.predict(X_train_transformed)
pred_test_ys=bnb.predict(X_test_transformed)

#accuracy
print("Train accuracy", accuracy_score(y,pred_train_ys))
print("Test accuracy", accuracy_score(y,pred_test_ys))

Multinomial has imporved the accuracy

# WORD EMBEDDING APPROACHES

1.  Word2vec-train our own vectors

2.  Glove-using pretrained vectors

**1.Glove**

In [4]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file='glove.6B.100d.txt'
word2vec_output_file='glove.6B.100d.w2vformat.txt'
glove2word2vec(glove_input_file,word2vec_output_file)

<ipython-input-4-d127d3bcd4f7>:5: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file,word2vec_output_file)


(400000, 100)

In [5]:
from gensim.models.keyedvectors import KeyedVectors
glove_model=KeyedVectors.load_word2vec_format("glove.6B.100d.w2vformat.txt",binary=False)

**training our own word vectors on the data(test and train)**


In [ ]:
x_comb=np.concatenate([X,X_test])
len(X_comb)

In [ ]:
print(X_comb[6000])

In [ ]:
w2v=word2vec.Word2Vec(X_comb,window=2,min_count=2,sg=1,size=200)
w2v.most_similar("future",topn=5)

**2.Sentence vectors by averaging the word vectors**

In [ ]:
#converting singular word into sentence vector by averaging the word


def sent_vec_w2v(sent):   #passing ever word vector
    wv_res=np.zeros(w2v.vector_size)
    ctr=1
    for w in sent:
        if w in w2v:
            ctr+=1           #average of word vector
            wv_res+=w2v[w]
        wv_res=wv_res/ctr
        return wv_res

In [ ]:
#passing the test and train data through the above function

train_doc_vecs=[]
for doc in X:
    doc_words=[term for term in doc if term not in stop_words]
    train_doc_vecs.append(sent_vec_w2v(doc_words))
    
test_doc_vec=[]
for doc in X_test:
    doc_words=[term for term in doc if term not in stop_words]
    test_doc_vecs.append(sent_vec_w2v(doc_words))

**3. Logistic regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg=LogisticRegression(penalty='l1',random_state=42,C=8)

#fitting with training data and label
logreg.fit(train_docs_vecs,y)

#prediction on training and test data
pred_train_ys=logreg.predict(train_doc_vecs)
pred_test_ys=logreg.predict(test_doc_vecs)

We get same accuracy with reduced set of features in Mulinomial NB

Hence word vectors performance is better than Multinomial NB:
1.  Smaller model
2.  Lower time for modelling